In [ ]:
!pip install langchain langchain-community langchain-huggingface sentence-transformers chromadb tiktoken requests transformers accelerate
!pip install transformers accelerate

FINAL CODE PROTOTYPE EXPECTING MORE UPDATES

In [ ]:

topic = 'Virat'  #@param {type:"string"}

article_limit = 100 #@param {type:"number"}

import os
from google.colab import userdata
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.chains import RetrievalQA

api_key = userdata.get('MEDIASTACK_API_KEY')

url = f"http://api.mediastack.com/v1/news?access_key={api_key}&keywords={topic}&languages=en&limit={article_limit}"

print(f"Fetching {article_limit} news articles for topic: {topic}...")

response = requests.get(url)
response_data = response.json()

articles = []
if 'data' in response_data:
    for item in response_data['data']:
        if item.get('description') and len(item['description']) > 50:
            articles.append({
                "title": item['title'],
                "content": item['description'],
                "url": item['url']
            })

print(f"Successfully fetched {len(articles)} articles.")

if articles:
    docs = [Document(page_content=article['content'], metadata={'source': article['url']}) for article in articles]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )

    chunked_docs = text_splitter.split_documents(docs)

    print(f"\nOriginal number of documents: {len(docs)}")
    print(f"Number of chunks after splitting: {len(chunked_docs)}")
    print("\n--- Example Chunk ---")
    if chunked_docs:
        print(chunked_docs[0].page_content)
        print(f"\nSource: {chunked_docs[0].metadata['source']}")
else:
    print("\nNo articles were fetched. Please check your MediaStack API key and plan.")

if 'articles' in locals() and articles:
    docs = [Document(page_content=article['content'], metadata={'source': article['url']}) for article in articles]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )

    chunked_docs = text_splitter.split_documents(docs)

    print(f"Original number of documents: {len(docs)}")
    print(f"Number of chunks after splitting: {len(chunked_docs)}")
    print("\n--- Example Chunk ---")
    if chunked_docs:
        print(chunked_docs[0].page_content)
        print(f"\nSource: {chunked_docs[0].metadata['source']}")
else:
    print("The 'articles' variable was not found. Please re-run the previous cell to fetch the data.")

print("Initializing a free embedding model from Hugging Face...")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
print("Model initialized.")
print("Creating vector database. This may take a moment...")

vectorstore = Chroma.from_documents(
    documents=chunked_docs,
    embedding=embeddings
)

print("\nVector database created successfully!")
print(f"Number of vectors in the store: {vectorstore._collection.count()}")
print("Loading free text-generation model from Hugging Face...")
model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512
)

llm = HuggingFacePipeline(pipeline=pipe)
print("Model loaded successfully.")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

def handle_query():
    """
    Starts an interactive loop to ask questions, view headlines, or quit.
    """
    while True:
        user_input = input("Ask a question, type 'headlines' to see the news list, or type 'quit' to exit: ")

        if user_input.lower() == 'quit':
            print("Exiting NexusNews. Goodbye!")
            break

        elif user_input.lower() == 'headlines':
            print("\n--- Fetched Headlines ---")
            if 'articles' in locals() and articles:
                for i, article in enumerate(articles):
                    print(f"{i + 1}. {article['title']}")
                    print(f"   Link: {article['url']}")
            else:
                print("No articles were loaded or found.")
            print("\n" + "="*50 + "\n")

        else:
            print(f"\nQuestion: {user_input}\n")
            result = qa_chain.invoke({"query": user_input})

            print("--- Answer ---")
            print(result['result'])

            print("\n--- Sources ---")
            sources = {doc.metadata['source'] for doc in result['source_documents']}
            for source in sources:
                print(source)
            print("\n" + "="*50 + "\n")

handle_query()


Fetching 100 news articles for topic: Virat...
Successfully fetched 100 articles.

Original number of documents: 100
Number of chunks after splitting: 100

--- Example Chunk ---
Among captains with 200+ caps, only Ricky Ponting (67.90) has a better winning percentage than Virat Kohli (63.38).The post India will dominate cricket for the next 100 years because of Virat Kohli – Ambati Rayudu appeared first on Inside Sport India.

Source: https://www.insidesport.in/cricket/india-will-dominate-cricket-for-the-next-100-years-because-of-virat-kohli-ambati-rayudu/
Original number of documents: 100
Number of chunks after splitting: 102

--- Example Chunk ---
Among captains with 200+ caps, only Ricky Ponting (67.90) has a better winning percentage than Virat Kohli (63.38).The post India will dominate cricket for the next 100 years because of Virat Kohli – Ambati Rayudu appeared first on Inside Sport India.

Source: https://www.insidesport.in/cricket/india-will-dominate-cricket-for-the-next-100-y

Device set to use cpu


Model loaded successfully.
Ask a question, type 'headlines' to see the news list, or type 'quit' to exit: Top recent news on Virat Kohli

Question: Top recent news on Virat Kohli

--- Answer ---
Not Virat Kohli Or Rohit Sharma! This Batter Is Most Successful Captain In Asia Cup History&nbsp;&nbsp;News18 Team India star Virat Kohli is regarded as one of the greatest batters of all time, particularly in the white-ball formats.

--- Sources ---
https://www.sportskeeda.com/cricket/virat-kohli-vs-brian-lara-comparing-stats-odi-cricket
https://www.sportskeeda.com/cricket/news-virat-kohli-clicks-photo-young-fan-london-park-eng-vs-ind-2025-3rd-test-in-picture
https://news.google.com/rss/articles/CBMi3AFBVV95cUxPc2JTclZzVjJObFk2cnFIcWl0NjY2bUNVQXZFXzBMVnB5YWw5RlpXY21KVm1zMTZGNzZTbngtU0U5clRnNHhoS2MzSWx5dVR4R3B2dUxzbWlIbjJUcGgwRi1KMEhtMDZMZWtCXzJxcXFIR0w2amZKcnF2aG1SYWt1MllCMlJnc041Q0JzU29yMjB5Sm9yTXBmSDRTZzNQS0hHZHpxSmdoZ2pTWmV4YUtEOHpBUEtNMDVVVENsT0I2eUM4cHpIOGNlUmM3WVVKZFpfZzJWNmMyUGVLc1lM?oc